# Detección de armas y otros objetos de interés a partir de vídeo

## Cargar librerías

In [20]:
import numpy as np
import cv2
import ipywidgets
import torch
import models
import utils
import numpy as np
import PIL.Image
import io
from ipywebrtc import ImageRecorder, CameraStream, VideoRecorder, VideoStream

## Cargar la red YOLOv5 entrenada con el dataset de armas

In [21]:
# Cargar modelo
yolov5 = torch.hub.load('.', 'custom', path='weights/best.pt', source='local', force_reload=True)

YOLOv5 🚀 7dde989 Python-3.8.13 torch-1.11.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1650 Ti with Max-Q Design, 3912MiB)

Fusing layers... 
Model summary: 213 layers, 1767283 parameters, 0 gradients
Adding AutoShape... 


## Preparar el vídeo y barra deslizante

In [23]:
# Acceder al video
video = VideoStream.from_file('videos/knife.mp4')
image_recorder = ImageRecorder(stream=video)
image_recorder.autosave = False
image_recorder.download()
video

VideoStream(video=Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x16Ymoov\x00\x00…

In [6]:
# Colores aleatorios para cada clase
np.random.seed(42)
colors = np.random.randint(0, 255, (6, 3), dtype=np.uint8)

# Barra deslizante
slider = ipywidgets.FloatSlider(
    value=0.3,
    min=0,
    max=1.0,
    step=0.05,
    description='Confianza:',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=True,
    readout_format='.2f',
)

## Procesamiento de imagen: Aplicar YOLOv5 y pintar detecciones

In [7]:
out = ipywidgets.Image()

# Función de procesado de imagen
def process_image(_):
    
    # Carga la imagen desde image_recorder
    im = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    frame = np.array(im)[...,:3].astype(np.uint8)
    
    
    # Procesa la imagen en la red YOLOv5
    results = yolov5(frame)
    
    
    # Dibuja los rectángulos de las detecciones resultantes de la red
    for index, row in results.pandas().xyxy[0].iterrows():
        x1 = int(row['xmin'])
        x2 = int(row['xmax'])
        y1 = int(row['ymin'])
        y2 = int(row['ymax'])
        label = '{} {:.1f}'.format(row['name'],row['confidence'])
        color = [int(c) for c in colors[int(row['class'])]]

        # Caja de la detección
        img = cv2.rectangle(frame, (x1, y1), (x2, y2), (color[0],color[1],color[2]), 2)

        # Cálculo tamaño texto
        (w, h), _ = cv2.getTextSize(
            label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)

        # Caja de texto
        img = cv2.rectangle(img, (x1, y1 - 20), (x1 + w, y1), (color[0],color[1],color[2]), -1)
        #img = cv2.putText(img, label, (x1, y1 - 5),
        #                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 1)

        # Escribir clase
        img = cv2.putText(img, label, (x1, y1),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    
    
    # Devuelve la imagen al formato adecuado para poder mostrarla
    f = io.BytesIO()
    PIL.Image.fromarray(frame).save(f, format='png')
    out.value = f.getvalue()

# Aplica la función de procesar imagen repetidamente
def process_video(_):
    if stop_process:
        return
    process_image(_)
    image_recorder.recording = True

In [8]:
# Función de barra deslizante
def handle_slider(_):
    yolov5.conf = slider.value # Cambia la confianza de la red YOLOv5 en función de la posición del slider
    process_image(_)    
    

# Aplica el manejador a la barra deslizante
slider.observe(handle_slider, names='value')

## Ejecutar programa detección de armas a partir de vídeo

In [18]:
stop_process = False
image_recorder.image.observe(process_video, names=['value'])
image_recorder.recording = True
ipywidgets.HBox([image_recorder, out, slider])

## Cambiar video

In [ ]:
#video = VideoStream.from_file('videos/knife.mp4')
video = VideoStream.from_file('videos/knife-gun.mp4')
image_recorder = ImageRecorder(stream=video)

### Parar ejecución vídeo

In [ ]:
stop_process = True